In [25]:
import os
import zipfile
import shutil

sturcture_zip_path = '/mnt/sdc/lanwei/software/cPEPmatch/5w59/colab/output/'
top_pose_path = '/mnt/sdc/lanwei/software/cPEPmatch/5w59/colab/cyclopep/'
extracted_folder_base = '/mnt/sdc/lanwei/software/cPEPmatch/5w59/colab/result/'
cyclic = '/mnt/sdc/lanwei/software/cPEPmatch/5w59/colab/cycol/'

In [10]:
for file_name in os.listdir(sturcture_zip_path):
    if file_name.endswith('.zip'):
        file_path = os.path.join(sturcture_zip_path, file_name)

        folder_name = os.path.splitext(file_name)[0]
        extracted_folder_path = os.path.join(extracted_folder_base, folder_name)
        os.makedirs(extracted_folder_path, exist_ok=True)
        with zipfile.ZipFile(sturcture_zip_path+'/'+file_name, 'r') as zip_ref:
            zip_ref.extractall(extracted_folder_path)        


In [23]:
import os
import zipfile
import shutil

sturcture_zip_path = '/mnt/sdc/lanwei/software/cPEPmatch/5w59/colab/output/'
top_pose_path = '/mnt/sdc/lanwei/software/cPEPmatch/5w59/colab/cyclopep/'
extracted_folder_base = '/mnt/sdc/lanwei/software/cPEPmatch/5w59/colab/result/'
cyclic = '/mnt/sdc/lanwei/software/cPEPmatch/5w59/colab/cycol/'

for file_name in os.listdir(sturcture_zip_path):
    if file_name.endswith('.zip'):
        file_path = os.path.join(sturcture_zip_path, file_name)

        folder_name = os.path.splitext(file_name)[0]
        extracted_folder_path = os.path.join(extracted_folder_base, folder_name)
        os.makedirs(extracted_folder_path, exist_ok=True)
        with zipfile.ZipFile(sturcture_zip_path+'/'+file_name, 'r') as zip_ref:
            zip_ref.extractall(extracted_folder_path)       

sequences = {}
for file_name in os.listdir(extracted_folder_base):
    for files in os.listdir(extracted_folder_base+file_name):
        if '_relaxed_rank_' in files and '.pdb' in files:
            file = extracted_folder_base+file_name+'/'+files
            #print(file)
            with open(file) as f:
                lines = f.readlines()
            #print(file)
            # 找到SG原子所在行
            sg_atoms = []
            for line in lines:
                if line.startswith('ATOM') and line[21] =='B' and line[12:16].strip() == 'SG':
                    sg_atoms.append(line)
            #print(sg_atoms)        
            if len(sg_atoms) == 2:
                # 提取SG坐标
                x1, y1, z1 = [float(sg_atoms[0][30:38]), float(sg_atoms[0][38:46]), float(sg_atoms[0][46:54])]
                x2, y2, z2 = [float(sg_atoms[1][30:38]), float(sg_atoms[1][38:46]), float(sg_atoms[1][46:54])]
                
                # 计算距离
                dist = ((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)**0.5
                
                if dist < 2.05:
                    part = files.split('_')
                    seqence = part[0]
                    rank = int(part[3])
                    if seqence not in sequences or rank < sequences[seqence][0]:
                        sequences[seqence] = (rank, file)


for seq, (rank, filepath) in sequences.items():
    outfile = os.path.join(cyclic, seq + '.pdb')
    os.rename(filepath, outfile)





{'CTDKEPPC': (1,
  '/mnt/sdc/lanwei/software/cPEPmatch/5w59/colab/result/CTDKEPPC.result/CTDKEPPC_relaxed_rank_001_alphafold2_multimer_v3_model_5_seed_000.pdb'),
 'CYNEYCPLG': (1,
  '/mnt/sdc/lanwei/software/cPEPmatch/5w59/colab/result/CYNEYCPLG.result/CYNEYCPLG_relaxed_rank_001_alphafold2_multimer_v3_model_5_seed_000.pdb'),
 'CSFEFC': (1,
  '/mnt/sdc/lanwei/software/cPEPmatch/5w59/colab/result/CSFEFC.result/CSFEFC_relaxed_rank_001_alphafold2_multimer_v3_model_2_seed_000.pdb'),
 'CGTPFC': (2,
  '/mnt/sdc/lanwei/software/cPEPmatch/5w59/colab/result/CGTPFC.result/CGTPFC_relaxed_rank_002_alphafold2_multimer_v3_model_1_seed_000.pdb'),
 'CHPPYWTC': (1,
  '/mnt/sdc/lanwei/software/cPEPmatch/5w59/colab/result/CHPPYWTC.result/CHPPYWTC_relaxed_rank_001_alphafold2_multimer_v3_model_3_seed_000.pdb'),
 'CHPQENEC': (3,
  '/mnt/sdc/lanwei/software/cPEPmatch/5w59/colab/result/CHPQENEC.result/CHPQENEC_relaxed_rank_003_alphafold2_multimer_v3_model_3_seed_000.pdb'),
 'CNSINC': (1,
  '/mnt/sdc/lanwei/sof

In [26]:
import os
import zipfile
from collections import defaultdict

def extract_zip_files(zip_folder, extracted_folder_base):
    for file_name in os.listdir(zip_folder):
        if file_name.endswith('.zip'):
            file_path = os.path.join(zip_folder, file_name)
            folder_name = os.path.splitext(file_name)[0]
            extracted_folder_path = os.path.join(extracted_folder_base, folder_name)
            os.makedirs(extracted_folder_path, exist_ok=True)
            
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(extracted_folder_path)

def find_sequences_and_rename(extracted_folder_base, cyclic_folder):
    sequences = defaultdict(lambda: (float('inf'), None))

    for root, dirs, files in os.walk(extracted_folder_base):
        for file_name in files:
            if '_relaxed_rank_' in file_name and file_name.endswith('.pdb'):
                file_path = os.path.join(root, file_name)

                # Extract SG coordinates
                with open(file_path) as f:
                    lines = f.readlines()

                sg_atoms = [line for line in lines if line.startswith('ATOM') and line[21] == 'B' and line[12:16].strip() == 'SG']

                if len(sg_atoms) == 2:
                    x1, y1, z1 = [float(sg_atoms[0][30:38]), float(sg_atoms[0][38:46]), float(sg_atoms[0][46:54])]
                    x2, y2, z2 = [float(sg_atoms[1][30:38]), float(sg_atoms[1][38:46]), float(sg_atoms[1][46:54])]
                    dist = ((x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2) ** 0.5

                    if dist < 2.05:
                        parts = file_name.split('_')
                        sequence = parts[0]
                        rank = int(parts[3])

                        if rank < sequences[sequence][0]:
                            sequences[sequence] = (rank, file_path)

    for seq, (rank, file_path) in sequences.items():
        outfile = os.path.join(cyclic_folder, f'{seq}.pdb')
        os.rename(file_path, outfile)

if __name__ == "__main__":
    sturcture_zip_path = '/mnt/sdc/lanwei/software/cPEPmatch/5w59/colab/output/'
    extracted_folder_base = '/mnt/sdc/lanwei/software/cPEPmatch/5w59/colab/result/'
    cyclic_folder = '/mnt/sdc/lanwei/software/cPEPmatch/5w59/colab/cycol/'

    extract_zip_files(sturcture_zip_path, extracted_folder_base)
    find_sequences_and_rename(extracted_folder_base, cyclic_folder)
